In [ ]:
#initial scrapping
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
# Define base URL
base_url = 'https://www.kilimall.co.ke/category/computer?id=100000357&form=category-hot&page='

# Initialize an empty list to hold product data
products = []

# Loop through pages 1 to 126
for page in range(1, 127):  # 127 because the range is exclusive of the end
    # Construct the URL for the current page
    url = base_url + str(page)
    
    # Fetch the page content
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}")
        continue

    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all ads on the page
    ads = soup.find_all("div", class_="inner-listing")

    # Extract details for each ad on the page
    for ad in ads:
        # Extract the necessary information with error handling
        product_title = ad.find('p', class_='product-title').get_text(strip=True) if ad.find('p', class_='product-title') else 'N/A'
        product_link = ad.find('a', href=True)['href'] if ad.find('a', href=True) else 'N/A'
        product_price = ad.find('div', class_='product-price').get_text(strip=True) if ad.find('div', class_='product-price') else 'N/A'
        image_tag = ad.find('img', data_src=True)
        image_url = image_tag['data-src'] if image_tag else 'N/A'
        reviews_count = ad.find('span', class_='reviews').get_text(strip=True) if ad.find('span', class_='reviews') else 'N/A'
        dispatch_status = ad.find('span', class_='tag-name').get_text(strip=True) if ad.find('span', class_='tag-name') else 'N/A'
        
        # Append the extracted data to the products list as a dictionary
        products.append({
            'Product Title': product_title,
            'Product Link': f"https://www.kilimall.co.ke{product_link}" if product_link != 'N/A' else 'N/A',
            'Price': product_price,
            'Image URL': image_url,
            'Reviews Count': reviews_count,
            'Dispatch Status': dispatch_status
        })

    # Optional: Add a short delay to avoid overloading the server
    time.sleep(1)

    # Print progress
    print(f"Page {page} processed")

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(products)

# Display the DataFrame or save it to a CSV
df
# df.to_csv('kilimall_products.csv', index=False)  # Uncomment to save as CSV


In [ ]:
#Remove all the non-numeric characters on the 'Price' column and convert it to integer
df['Price'] = df['Price'].str.replace(r'\D', '', regex=True).astype(int)


In [ ]:
import numpy as np

# Define conditions for processor types in 'Product Title' column
conditions = [
    (df['Product Title'].str.contains('i7', case=False, na=False)),
    (df['Product Title'].str.contains('i5', case=False, na=False)),
    (df['Product Title'].str.contains('i3', case=False, na=False)),
    (df['Product Title'].str.contains('Core 2 Duo', case=False, na=False)),
    (df['Product Title'].str.contains('Duo Core|Dual Core', case=False, na=False)),
    (df['Product Title'].str.contains('Pentium 6', case=False, na=False)),
    (df['Product Title'].str.contains('Pentium', case=False, na=False)),
    (df['Product Title'].str.contains('Celeron|YOGA', case=False, na=False)),
    (df['Product Title'].str.contains('Intel AMD 2ND GEN', case=False, na=False)),
    (df['Product Title'].str.contains('Macbook Air', case=False, na=False))  # New condition for Macbook Air
]

# Define corresponding values for each condition, ensuring 'Macbook Air' maps to 'i5'
values = ['i7', 'i5', 'i3', 'Core 2 Duo', 'Duo Core', 'Pentium 6', 'Pentium', 'Celeron', 'AMD Ryzen', 'i5']

# Create the 'Processor' column with np.select, defaulting to None if no match is found
df['Processor'] = np.select(conditions, values, default=None)

# Verify the column creation
print(df[['Product Title', 'Processor']].head())  # Displaying first few rows for verification




In [ ]:


# updated more rows for 'Processor' field
conditions = [
    (df['Product Title'].str.contains('Lenovo ThinkCenter 10B1', case=False, na=False)),
    (df['Product Title'].str.contains('Macbook', case=False, na=False)),
    (df['Product Title'].str.contains('i3', case=False, na=False)),
    (df['Product Title'].str.contains('Core 2 Duo', case=False, na=False)),
    (df['Product Title'].str.contains('Duo Core|Dual Core', case=False, na=False)),
    (df['Product Title'].str.contains('Pentium 6', case=False, na=False)),
    (df['Product Title'].str.contains('Pentium', case=False, na=False)),
    (df['Product Title'].str.contains('Celeron|Lenovo X131E', case=False, na=False)),
    (df['Product Title'].str.contains('RYZEN', case=False, na=False)),
    (df['Product Title'].str.contains('Macbook Air', case=False, na=False) & df['Product Title'].str.contains('M2', case=False, na=False)),  # Macbook Air with M2
    (df['Product Title'].str.contains('Macbook Air', case=False, na=False) & df['Product Title'].str.contains('M1', case=False, na=False)),  # Macbook Air with M1
    (df['Product Title'].str.contains('Macbook', case=False, na=False) & df['Product Title'].str.contains('M2', case=False, na=False)),  # Macbook with M2
    (df['Product Title'].str.contains('Macbook', case=False, na=False) & df['Product Title'].str.contains('M1', case=False, na=False))   # Macbook with M1
]

# Define corresponding values for each condition
values = ['i3', 'Apple M1 or M2 chip', 'i3', 'Core 2 Duo', 'Duo Core', 'Pentium 6', 'Pentium', 'Celeron', 'AMD Ryzen', 
          'Apple M2 chip', 'Apple M1 chip', 'Apple M2 chip', 'Apple M1 chip']





In [ ]:
# Remove rows where 'Product Title' contains 'tablet' or phone
df = df[~df['Product Title'].str.contains('Tablet', case=False, na=False)]
df = df[~df['Product Title'].str.contains('Phone', case=False, na=False)]
df = df[~df['Product Title'].str.contains('Android', case=False, na=False)]
df = df[~df['Product Title'].str.contains('Tab', case=False, na=False)]



In [ ]:
# Create a new 'Storage' column, setting it to 'SSD' if 'Product Title' contains 'MacBook'
df['Storage'] = np.where(df['Product Title'].str.contains('MacBook', case=False, na=False), 'SSD', None)

# Verify the column creation
print(df[['Product Title', 'Storage']].head())  # Displaying first few rows for verification


In [ ]:
df['Storage'] = np.where(df['Product Title'].str.contains('SSD', case=False, na=False), 'SSD', df['Storage'])


In [ ]:
#Storage Type

# Create 'Storage' column, assigning 'SSD' if 'Product Title' contains 'MacBook' or 'SSD'
df['Storage'] = np.where(
    df['Product Title'].str.contains('MacBook', case=False, na=False) | 
    df['Product Title'].str.contains('SSD', case=False, na=False),
    'SSD', None)

# S Update 'Storage' to 'HDD' for rows where 'Storage' is still None
df['Storage'] = np.where(df['Storage'].isna(), 'HDD', df['Storage'])




In [ ]:
#Remove all the non-numeric characters on the 'Reviews Count' column and convert it to integer
df['Reviews Count'] = df['Reviews Count'].str.replace(r'\D', '', regex=True).astype(int)

In [ ]:
#Upload the Dataframe into kilimall_products.xlsx excel file
df=pd.read_excel('kilimall_products.xlsx')